# sports-book-manager

## Example 1:
### Using the BookScraper class

Importing the scraper class and setting the domain and directory paths.

In [1]:
import sports_book_manager.book_scrape_class as bs
PointsBet = bs.BookScraper(domain=r'https://nj.pointsbet.com/sports',
                                          directories={'NHL':r'/ice-hockey/NHL',
                                                       'NBA':r'/basketball/NBA'
                                                      }
                                         )

Setting the element values for the scraper. Lines and Odds data are contained in a class that other data is stored in as well, so parent values need to be set for them. driver_path is needed only if Selenium cannot find the chromedriver using webdriver.Chrome().

In [2]:
PointsBet.update_html_elements(driver_path=..., ancestor_container='.facf5sk',
                               event_row=".f1oyvxkl", team_class='.fji5frh.fr8jv7a.f1wtz5iq', line_class='.fsu5r7i',
                               line_parent_tag='button', line_parent_attr='data-test', line_parent_val='Market0OddsButton',
                               odds_class='.fheif50', odds_parent_tag='button', odds_parent_attr='data-test',
                               odds_parent_val='Market0OddsButton')

Retrieving the PointsBet odds in a pandas dataframe.

In [3]:
data = PointsBet.retrieve_sports_book('NBA')
data

,Teams,Lines,Odds
0,Los Angeles Clippers,+6.5,-110
1,Cleveland Cavaliers,-6.5,-110
2,Denver Nuggets,+3,-110
3,Philadelphia 76ers,-3,-110
4,Portland Trail Blazers,+12,-110
5,Atlanta Hawks,-12,-110
6,Charlotte Hornets,-8.5,-110
7,Oklahoma City Thunder,+8.5,-110
8,Washington Wizards,+12.5,-110
9,Golden State Warriors,-12.5,-110


## Example 2:
### Using sports-book-manager functions to compare sports book wagers to model outputs

First we will pull an example of hockey wagers from the data folder in the sports-book-wager package.

In [4]:
from os import path
import sports_book_manager
import pandas as pd
data_path = path.join(path.abspath(sports_book_manager.__file__), '..', 'data')
df = pd.read_csv(f'{data_path}\\hockey_odds.csv')
df

,Teams,Lines,Odds
0,Colorado Avalanche,-1.5,120
1,New Jersey Devils,1.5,-140
2,Florida Panthers,-1.5,195
3,Pittsburgh Penguins,1.5,-245
4,Vegas Golden Knights,-1.5,145
5,Philadelphia Flyers,1.5,-176
6,Arizona Coyotes,1.5,-176
7,Detroit Red Wings,-1.5,145
8,Seattle Kraken,1.5,-110
9,Toronto Maple Leafs,-1.5,-110


For implied_probability_calculator and model_probability to work the values must be numeric. This is a good practice to follow as we don't what data type the scraper will set the data to.

In [5]:
df['Odds'] = pd.to_numeric(df['Odds'])
df['Lines'] = pd.to_numeric(df['Lines'])

Using pandas lambda function we can perform the implied_probability_calculator function to every row to create a new column.

In [6]:
import sports_book_manager.implied_probability_calculator as ipc
df['implied_probability'] = df.apply(lambda x: ipc.implied_probability_calculator(x['Odds']), axis = 1)
df

,Teams,Lines,Odds,implied_probability
0,Colorado Avalanche,-1.5,120,0.4545
1,New Jersey Devils,1.5,-140,0.5833
2,Florida Panthers,-1.5,195,0.3390
3,Pittsburgh Penguins,1.5,-245,0.7101
4,Vegas Golden Knights,-1.5,145,0.4082
5,Philadelphia Flyers,1.5,-176,0.6377
6,Arizona Coyotes,1.5,-176,0.6377
7,Detroit Red Wings,-1.5,145,0.4082
8,Seattle Kraken,1.5,-110,0.5238
9,Toronto Maple Leafs,-1.5,-110,0.5238


Getting an example of model outputs from the data folder in the sports-book-wager package and merging the two dataframes together.

In [7]:
model = pd.read_csv(f'{data_path}\\model_output_example.csv')
df_join = df.join(model, lsuffix='Teams', rsuffix='team')
df_join = df_join.drop(columns=['team'])

We use the lambda function again to perform the model_probability calculations which evaluate the model's expected probability of a team winning at the margin set with the market line.

In [8]:
import sports_book_manager.model_probability as mp
df_join['model_probability'] = df_join.apply(lambda x: mp.model_probability(x['mean_win_margin'],
                                                                         x['sd'], x['Lines']), axis = 1)
df_join

,Teams,Lines,Odds,implied_probability,opponent,mean_win_margin,sd,model_probability
0,Colorado Avalanche,-1.5,120,0.4545,New Jersey Devils,1.80,0.56,0.703922
1,New Jersey Devils,1.5,-140,0.5833,Colorado Avalanche,-1.80,0.56,0.296078
2,Florida Panthers,-1.5,195,0.3390,Pittsburgh Penguins,1.68,0.86,0.582894
3,Pittsburgh Penguins,1.5,-245,0.7101,Florida Panthers,-1.68,0.86,0.417106
4,Vegas Golden Knights,-1.5,145,0.4082,Philadelphia Flyers,1.72,0.43,0.695544
5,Philadelphia Flyers,1.5,-176,0.6377,Vegas Golden Knights,-1.72,0.43,0.304456
6,Arizona Coyotes,1.5,-176,0.6377,Detroit Red Wings,-1.21,0.38,0.777315
7,Detroit Red Wings,-1.5,145,0.4082,Arizona Coyotes,1.21,0.38,0.222685
8,Seattle Kraken,1.5,-110,0.5238,Toronto Maple Leafs,-1.65,0.51,0.384334
9,Toronto Maple Leafs,-1.5,-110,0.5238,Seattle Kraken,1.65,0.51,0.615666


We can then format our pandas dataframe to compare the market's expected probability to the model's and find the bets the model feels best about.

In [9]:
df_join['difference'] = df_join.apply(lambda x: (x['model_probability']) - x['implied_probability'], axis=1)
df_join = df_join.sort_values(by=['difference'], ascending=False)
df_join

,Teams,Lines,Odds,implied_probability,opponent,mean_win_margin,sd,model_probability,difference
16,Tampa Bay Lightning,-1.5,145,0.4082,Winnipeg Jets,1.68,0.33,0.707280,0.299080
4,Vegas Golden Knights,-1.5,145,0.4082,Philadelphia Flyers,1.72,0.43,0.695544,0.287344
0,Colorado Avalanche,-1.5,120,0.4545,New Jersey Devils,1.80,0.56,0.703922,0.249422
2,Florida Panthers,-1.5,195,0.3390,Pittsburgh Penguins,1.68,0.86,0.582894,0.243894
11,St. Louis Blues,-1.5,-110,0.5238,Ottawa Senators,1.95,0.68,0.745939,0.222139
6,Arizona Coyotes,1.5,-176,0.6377,Detroit Red Wings,-1.21,0.38,0.777315,0.139615
18,Anaheim Ducks,1.5,-245,0.7101,Chicago Blackhawks,-0.73,0.85,0.817501,0.107401
9,Toronto Maple Leafs,-1.5,-110,0.5238,Seattle Kraken,1.65,0.51,0.615666,0.091866
14,Dallas Stars,1.5,-236,0.7024,Nashville Predators,-0.97,0.68,0.782131,0.079731
12,New York Rangers,1.5,-180,0.6429,Minnesota Wild,-1.20,0.56,0.703922,0.061022
